In [76]:
from binarytree import tree, Node
import math

In [22]:
def parse_tree(s):
    stack = []
    for c in s:
        if c == "[":
            stack.append(c)
        elif c == ",":
            pass
        elif c == "]":
            t2 = stack.pop()
            t1 = stack.pop()
            _ = stack.pop()
            assert _ == "["
            t = Node(0)
            t.left = t1
            t.right = t2
            stack.append(t)
        else:
            stack.append(Node(int(c)))
        # for b in stack:
        #     print(b)
    assert len(stack) == 1
    return stack[0]

In [52]:
def find_left_neighbour(root, node):
    leaves = [n for n in root.postorder if n.left is None]
    inds = [i for i,v in enumerate(leaves) if v == node]
    assert len(inds) == 1
    i = inds[0]
    neighbour = leaves[i-1] if i > 0 else None
    return neighbour
    
def find_right_neighbour(root, node):
    leaves = [n for n in root.postorder if n.left is None]
    inds = [i for i,v in enumerate(leaves) if v == node]
    assert len(inds) == 1
    i = inds[0]
    neighbour = leaves[i+1] if i < len(leaves)-1 else None
    return neighbour

In [101]:
def tree_add(root, another):
    if root is None:
        return another
    a = Node(0)
    a.left = root
    a.right = another
    return a

In [108]:
def tree_explode(root):
    while root.height > 4:
        for parent in root.levels[4]:
            if parent.left is None:
                continue
            else:
                # assert: left / right must have value at same time.
                node_l = parent.left
                nb_l = find_left_neighbour(root, node_l)
                if nb_l:
                    nb_l.value += node_l.value
                
                node_r = parent.right
                nb_r = find_right_neighbour(root, node_r)
                if nb_r:
                    nb_r.value += node_r.value
                
                # reset this parent = 0
                parent.left  = None
                parent.right = None
                parent.value = 0
            # print(root)
    return root
                

In [99]:
def tree_split(root):
    leaves = [n for n in root.postorder if n.left is None]
    for leaf in leaves:
        if leaf.value > 9:
            # print(leaf, leaf.left, leaf.right)
            leaf.left = Node(math.floor(leaf.value/2))
            leaf.right = Node(leaf.value - leaf.left.value)
            leaf.value = 0
            # only split first then check explode
            break
    else:
        # no leaf ever
        return root
    root = tree_explode(root)
    if root.max_node_value > 9:
        return tree_split(root)
    else:
        return root
    

In [105]:
def tree_sum(root):
    if root.left is None:
        return root.value
    else:
        return 3 * tree_sum(root.left) + 2 * tree_sum(root.right)

In [113]:
a = "[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]"
t = parse_tree(a)
print(t)

# test left or right neighbours

node = t.left.left.right.left
print(node)
left_node = find_left_neighbour(t, node)
assert left_node.val == 0
right_node = find_right_neighbour(t, node)
assert right_node.val == 5


            ______0______________
           /                     \
    ______0__               ______0__
   /         \             /         \
  0__         0         __0__         0
 /   \       / \       /     \       / \
0     0     0   0     0       0     9   5
     / \             / \     / \
    4   5           4   5   2   6


4



In [114]:
# test explode and split
s = "[[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]]"
t = parse_tree(s)
print(t)
print("--------------test explode--------------------")
t = tree_explode(t)
print(t)
print("--------------test split --------------------")
t = tree_split(t)
print(t)


                ______________0__
               /                 \
            __0__                 0
           /     \               / \
        __0       0______       1   1
       /   \     /       \
    __0     4   7       __0
   /   \               /   \
  0     4             0     9
 / \                 / \
4   3               8   4

--------------test explode--------------------

            ____________0__
           /               \
        __0___              0
       /      \            / \
    __0       _0__        1   1
   /   \     /    \
  0     4   15     0
 / \              / \
0   7            0   13

--------------test split --------------------

            ______________0__
           /                 \
        __0______             0
       /         \           / \
    __0         __0__       8   1
   /   \       /     \
  0     4     0       0
 / \         / \     / \
0   7       7   8   6   0



In [112]:
# test combine
ss = """[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]"""

root = None
for s in ss.split("\n"):
    t = parse_tree(s)
    print(t)
    root = tree_split(tree_explode(tree_add(root, t)))
    print(root)
    
print(tree_sum(root))


            ______________0__________
           /                         \
    ______0______               ______0______
   /             \             /             \
  0__           __0__         0__           __0
 /   \         /     \       /   \         /   \
0     0       0       0     4     0       0     2
     / \     / \     / \         / \     / \
    5   8   1   7   9   6       1   2   1   4


            ______________0__________
           /                         \
    ______0______               ______0______
   /             \             /             \
  0__           __0__         0__           __0
 /   \         /     \       /   \         /   \
0     0       0       0     4     0       0     2
     / \     / \     / \         / \     / \
    5   8   1   7   9   6       1   2   1   4


            __0__
           /     \
    ______0       0______
   /       \     /       \
  0__       4   5       __0
 /   \                 /   \
5     0               0     0
  

In [109]:
# Q1
ss = """[[[[7,7],2],[[9,2],4]],[[[9,1],5],[[9,6],[6,4]]]]
[[[2,0],[8,[9,4]]],[[1,0],0]]
[8,[[[9,5],7],[9,7]]]
[[[[1,3],[1,8]],[[8,8],5]],[[7,[4,0]],2]]
[[[[7,8],3],[9,3]],5]
[[5,[[9,3],4]],[[[0,1],7],[6,[8,3]]]]
[[[[1,6],[4,1]],[0,3]],[9,[[4,3],[3,2]]]]
[[[[7,9],8],4],[[[9,0],1],[[9,8],[0,5]]]]
[[[8,7],[6,1]],[[[1,3],[6,6]],[5,[4,5]]]]
[[[[9,8],[2,1]],[[2,3],2]],5]
[6,3]
[[[9,1],6],[[[7,1],[6,8]],[[8,3],[6,4]]]]
[4,[[8,[7,1]],[8,[7,2]]]]
[[[1,6],[9,[0,8]]],[[6,7],[2,[4,5]]]]
[[[[1,8],[9,2]],5],[[[8,6],[2,1]],[0,6]]]
[[[[0,2],4],[4,[3,6]]],7]
[[[[7,5],5],7],[[[6,0],4],[5,0]]]
[[2,1],[[[3,0],[1,4]],7]]
[[[[9,4],[2,8]],9],[[[9,1],[7,3]],[1,[2,1]]]]
[[[[4,2],3],[6,4]],[[6,0],[1,5]]]
[2,6]
[[4,6],[[2,2],[3,0]]]
[[[[6,4],[0,7]],[0,8]],[[[6,7],2],7]]
[[8,[[4,0],[8,4]]],1]
[[3,[6,6]],[[[6,4],[1,5]],[4,0]]]
[[[9,5],[5,[4,0]]],[[1,[0,6]],[[5,8],0]]]
[[[[6,1],8],[3,7]],[[[6,4],0],[[4,8],4]]]
[[[[3,1],3],[[3,6],[3,8]]],[[[6,7],0],2]]
[[4,1],[[[4,8],7],[3,0]]]
[[[[0,6],[1,3]],[[0,8],[1,9]]],3]
[[0,[3,1]],[[[0,0],6],[[7,6],3]]]
[[6,[[5,4],7]],[8,[5,5]]]
[[[6,3],[[8,9],6]],2]
[9,[[8,3],7]]
[[[1,[3,0]],[[3,7],5]],[[5,8],[[3,7],[8,6]]]]
[[[[6,1],2],[[7,8],[3,9]]],[[[3,6],[6,8]],[5,5]]]
[[[[6,8],[7,1]],[8,1]],[[[1,6],9],[[3,3],[7,9]]]]
[[[[6,9],0],[5,6]],3]
[[[9,6],[[0,5],[2,0]]],[[[6,7],7],[2,6]]]
[[0,[5,8]],[[1,[4,6]],[4,6]]]
[[[[3,3],4],[0,1]],[[[6,5],0],[2,3]]]
[0,4]
[[5,5],[[[6,5],8],7]]
[[[[7,3],[9,1]],[[9,0],2]],[[7,[8,3]],[[9,5],[7,3]]]]
[[[[1,2],[7,7]],[9,0]],[0,7]]
[[[0,[8,6]],[1,3]],[[6,6],9]]
[[[0,2],[4,7]],0]
[[[9,[9,6]],1],[[[1,5],[1,7]],[[5,1],[8,1]]]]
[[[6,9],4],0]
[[[[4,9],6],5],[7,[3,[9,8]]]]
[[6,[6,[5,7]]],[0,[[7,4],8]]]
[[4,[5,0]],[2,3]]
[[[[8,6],9],[3,[1,2]]],[1,[8,[3,8]]]]
[[[8,4],[7,2]],9]
[[[[6,3],[6,2]],[2,[0,0]]],[[[6,4],[1,6]],[[3,5],6]]]
[7,[[[2,4],0],[9,[9,9]]]]
[[[9,2],8],[[2,[9,9]],[9,[7,4]]]]
[1,[[0,7],[[1,6],0]]]
[[[[5,5],4],8],[[9,[6,5]],[[7,4],7]]]
[[[[7,6],4],[8,4]],[2,[1,[5,1]]]]
[[[2,[1,2]],7],[7,[[9,9],3]]]
[1,[[3,[9,9]],[5,6]]]
[[3,[[1,8],4]],[[9,[6,9]],2]]
[[[2,[4,5]],[1,[9,0]]],[4,1]]
[[[7,[5,9]],[7,7]],[[3,[4,0]],[2,[0,0]]]]
[[[0,[9,8]],0],[8,[7,1]]]
[[[6,6],[0,[4,8]]],3]
[[1,[[8,2],[9,9]]],3]
[[2,[5,[6,7]]],[[5,3],3]]
[[2,[[5,0],[8,5]]],[[7,[0,5]],[[5,7],3]]]
[[[[9,4],[4,0]],[6,[7,8]]],[[7,6],1]]
[[0,2],6]
[[[7,5],[[7,4],[4,1]]],[3,[[6,6],[5,5]]]]
[[3,[[0,7],8]],[[1,7],[5,0]]]
[[9,[[9,7],[3,0]]],6]
[[[[7,9],2],[3,[5,4]]],[[[9,4],[5,8]],[[5,0],[4,2]]]]
[[[[4,3],6],7],[[2,6],[5,[0,1]]]]
[[1,[3,5]],[[4,[5,0]],1]]
[[[9,[3,9]],8],[9,[[2,9],[2,2]]]]
[[[0,[5,0]],[[4,4],3]],6]
[[[9,3],[[2,4],[8,4]]],[[[6,8],[3,6]],[[4,6],[1,2]]]]
[[[[8,2],[3,2]],[4,[1,1]]],[[[7,2],1],[[9,9],[0,5]]]]
[[[6,3],[[3,6],9]],[6,5]]
[8,[[[8,7],3],[4,3]]]
[[[[8,3],3],[[6,1],9]],[[[2,4],[5,9]],[[9,7],1]]]
[[[2,[6,4]],[[0,1],3]],[[[1,2],9],[4,7]]]
[7,9]
[[[3,[1,4]],5],[[4,[5,1]],8]]
[[[[7,6],4],0],[5,5]]
[[4,[[5,2],5]],[[[0,4],[6,1]],[[3,0],[4,9]]]]
[[[[8,6],[6,1]],9],[[[4,1],2],[[9,2],3]]]
[[[6,1],[[8,9],[9,0]]],[[[4,4],[3,0]],[[4,2],[9,9]]]]
[1,[[[8,8],3],7]]
[[1,[4,[6,8]]],[1,[7,0]]]
[6,[[3,[2,4]],[[4,5],[5,3]]]]
[8,[[9,[6,0]],[[2,5],0]]]
[[5,[0,8]],[[7,1],[[5,9],2]]]
[[[5,8],[[1,1],4]],[[0,1],[4,3]]]
[[3,[1,[7,3]]],[[[6,4],9],[[2,8],[0,1]]]]
[[[6,[2,5]],5],[0,[[5,3],[4,2]]]]"""

root = None
for s in ss.split("\n"):
    t = parse_tree(s)
    # print(s)
    root = tree_split(tree_explode(tree_add(root, t)))
    
print(tree_sum(root))

4323


In [110]:
# Q2
scores = []
for s1 in ss.split("\n"):
    for s2 in ss.split("\n"):
        t1 = parse_tree(s1)
        t2 = parse_tree(s2)
        # TODO: what if t1 == t2?
        scores.append(tree_sum(tree_split(tree_explode(tree_add(t1, t2)))))
    
# print(scores)
print(max(scores))

[4299, 4005, 4227, 4555, 4059, 4477, 4209, 4271, 4379, 3593, 2949, 4677, 3851, 4399, 4161, 3749, 4371, 3501, 4157, 4151, 2901, 3555, 4477, 3683, 4257, 4247, 4507, 4453, 3819, 3759, 3537, 4163, 3969, 3401, 4301, 4461, 4405, 3883, 4557, 3669, 3921, 2785, 4165, 4315, 4181, 4215, 3585, 4655, 3867, 4455, 4269, 3519, 3699, 3723, 4525, 3513, 4481, 3153, 4703, 4137, 4005, 3601, 4353, 3983, 4545, 3871, 3651, 3939, 4083, 4411, 4155, 2925, 4551, 4119, 3917, 4433, 4083, 3725, 4513, 3765, 4301, 4103, 4099, 4083, 4147, 4263, 2991, 3813, 3541, 4269, 4127, 4327, 3421, 3759, 3617, 3719, 4053, 4059, 4331, 3987, 3082, 1980, 1992, 2560, 2082, 2542, 2076, 2836, 2756, 2036, 1236, 2958, 1958, 2458, 2472, 1784, 2298, 1524, 2928, 2048, 1224, 1524, 2420, 2036, 2106, 2666, 2496, 2518, 1788, 2118, 1848, 2474, 2056, 1658, 2568, 2996, 2892, 1920, 2850, 1980, 1860, 1204, 2098, 3148, 2222, 2376, 1572, 2552, 1884, 2478, 2382, 1488, 2214, 2100, 2550, 1770, 2750, 1482, 2770, 2178, 2082, 1702, 2202, 2012, 2634, 2050, 198